In [188]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

import control as c
from control.matlab import * 

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import math
import ipywidgets as widgets


## Functies

In [302]:
#  Ontwerp het proces
def ontw_proces(z0, z1, p0, p1, p2, dodet):
    s = c.TransferFunction.s
    G = (z1*s + z0) / (p2*(s**2) + p1*s + p0)
    return G
    
# Ontwerp de regelaar    
def ontw_regelaar(Kp, Ti, Td):
    s = c.TransferFunction.s
    C = Kp * (1 + (1/(Ti*s)) + (Td*s))
    return C

# Visualiseer het stapantwoord van het proces
def f(z0, z1, p0, p1, p2, dodet):
    #ontw_proces(z0, z1, p0, p1, p2, dodet)
    s = c.TransferFunction.s
    G = (z1*s + z0) / (p2*(s**2) + p1*s + p0)
    T,y = c.step_response(G)
    plt.plot(T, np.ones(len(T)),'r--')
    Tdodet = T + dodet
    plt.plot(Tdodet, y)
    plt.xlim(0,10)
    plt.ylim(0,2)
    plt.xlabel('tijd[s]')
    plt.ylabel('amplitude[-]')
    plt.title('Open lus proces stapantwoord')
    print('Transferfunctie model: ',G)
    
# Visualiseer OL met stapantwoord
def vis_sys(C,G):
    sys = olsys(C,G)
    T,y = c.step_response(sys)
    plt.plot(T, np.ones(len(T)),'r--')
    plt.plot(T, y)
    plt.xlim(0,10)
    plt.ylim(0,2)
    plt.xlabel('tijd[s]')
    plt.ylabel('amplitude[-]')
    plt.title('Open lus proces stapantwoord')
    print('Systeem', sys)
    
# Visualiseer afstellen OL met stapantwoord
def vis_sys_tune(z0, z1, p0, p1, p2, dodet, Kp, Ti, Td):
#     G = ontw_proces(z0.value, z1.value, p0.value, p1.value, p2.value, dodet.value) 
#     C = ontw_regelaar(Kp.value, Ti.value, Td.value)
    G = ontw_proces(z0, z1, p0, p1, p2, dodet) 
    C = ontw_regelaar(Kp, Ti, Td)
    sys = c.series(C,G)
    T,y = c.step_response(sys)
    plt.plot(T, np.ones(len(T)),'r--')
    plt.plot(T, y)
    plt.xlim(0,10)
    plt.ylim(0,2)
    plt.xlabel('tijd[s]')
    plt.ylabel('amplitude[-]')
    plt.title('Open lus proces stapantwoord')
    print('Proces', G)
    print('Regelaar', C)
    print('Systeem', sys)    

# Maak een gesloten-lus systeem
def clsys(C,G,H):
    sys = c.parallel(c.series(C,G), H)
    return sys

# Analyseer het systeem
def systeemanalyse(sys):
    step(sys)
    impulse(sys)
    bode(sys)
    nyquist(sys)
    margin(sys)
    #sisotool(sys)
    return

# Geeft de procedure details weer
# def printprocedure(methode):
#     if methode == methodes[0]:
#     break

In [286]:
# procedures
class procedures:
    """Class met stappenplannen voor verschillende afstelregels"""
    def __init__(self):
        self.data = []
        
    def stp_intuitie(self):
        txt = '''
            Enkele vuistregels:
            De proportionele versterking vergroten zal de stabiliteit verlagen.
            * De error zal sneller verkleinen wanneer de integratietijd vergroot.
            * De T_i (= integratietijd) verlagen zal de stabiliteit verlagen.
            * De T_d (= afgeleidetijd) verhogen zal de stabiliteit verhogen.
            * Elimineer alle componenten die onnodige fasevertraging kunnen veroorzaken, want deze hebben een groot effect op de fasemarge. Enkele voorbeelden zijn filters en versterkers.
            * In een **cascade** systeem regelen we eerst de binnenlus en dan pas de buitenlus. De binnenlus werkt op een hogere frequentie en is vergelijkbaar met een laag-doorlaat filter.)

            '''


## Kies HOE je de regelaar wilt afstellen

In [287]:
# kies de methode waarmee je de PID wilt afstellen
methodes = ['Intuïtie', 
             'Empirische procedure',
            'Afstelkaart',
             'Transient respons Z-N', 
             'Transient respons CHR', 
             'Transient respons C-C',
             '']

choise = widgets.Select(
    options= methodes,
    rows=8,
    description='Methode:',
    disabled=False
)
choise

Select(description='Methode:', options=('Intuïtie', 'Empirische procedure', 'Afstelkaart', 'Transient respons …

## Stel het procesmodel in 

In [288]:
# parameters
minim = 0
maxim = 5
steps = 0.1
z0 = widgets.FloatSlider(description = 'z0', value=1, min=minim, max=maxim)
z1 = widgets.FloatSlider(description = 'z1', value=1, min=minim, max=maxim)
p0 = widgets.FloatSlider(description = 'p0', value=1, min=minim, max=maxim)
p1 = widgets.FloatSlider(description = 'p1', value=1, min=minim, max=maxim)
p2 = widgets.FloatSlider(description = 'p2',  value=1, min=minim, max=maxim)
dodet = widgets.FloatSlider(description = 'dodet', value=0, min=minim, max=maxim)

# Visualiseer
print('Proces:')
out = widgets.interactive_output(f,
                                 {'z0': z0,
                                  'z1': z1,
                                  'p0': p0,
                                 'p1': p1,
                                 'p2': p2,
                                 'dodet': dodet})
widgets.HBox([widgets.VBox([z0, z1, p0, p1, p2, dodet]), out])

Proces:


## Stel nu een P(I, D) controller samen

In [299]:
Kp = widgets.FloatSlider(description = 'Proportioneel', value=1, min=minim, max=maxim)
Ti = widgets.FloatSlider(description = 'Intergratietijd', value=1, min=0, max=100)
Td = widgets.FloatSlider(description = 'Afgeleidetijd', value=1, min=0, max=100)

print('PID regelaar versterking:')
out = widgets.interactive_output(ontw_regelaar,
                                 {'Kp': Kp, 'Ti': Ti, 'Td': Td})

widgets.HBox([widgets.VBox([Kp, Ti, Td]), out])

PID regelaar versterking:


## Intuïtief afstellen

In [303]:
z0 = widgets.FloatSlider(description = 'z0', value=1, min=minim, max=maxim)
z1 = widgets.FloatSlider(description = 'z1', value=1, min=minim, max=maxim)
p0 = widgets.FloatSlider(description = 'p0', value=1, min=minim, max=maxim)
p1 = widgets.FloatSlider(description = 'p1', value=1, min=minim, max=maxim)
p2 = widgets.FloatSlider(description = 'p2',  value=1, min=minim, max=maxim)
dodet = widgets.FloatSlider(description = 'dodet', value=0, min=minim, max=maxim)

Kp = widgets.FloatSlider(description = 'Proportioneel', value=1, min=minim, max=maxim)
Ti = widgets.FloatSlider(description = 'Intergratietijd', value=1, min=0, max=100)
Td = widgets.FloatSlider(description = 'Afgeleidetijd', value=1, min=0, max=100)

out = widgets.interactive_output(vis_sys_tune,
                                 {'z0': z0,
                                  'z1': z1,
                                  'p0': p0,
                                 'p1': p1,
                                 'p2': p2,
                                 'dodet': dodet,
                                  'Kp': Kp, 
                                  'Ti': Ti, 
                                  'Td': Td})

widgets.HBox([widgets.VBox([z0, z1, p0, p1, p2, dodet, Kp, Ti, Td]), out])

In [ ]:
# bronnen 
# https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html
# https://python-control.readthedocs.io/en/0.8.3/conventions.html1
# https://www.youtube.com/watch?v=8IYbdshUd9c
